In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import random
import re

In [2]:
cd=pd.read_csv("cars.csv")

Okuma

In [5]:
cd.to_csv("cars.csv",index=False)

Yazma

In [9]:
iller = [
    "adana", "adiyaman", "afyonkarahisar", "agri", "amasya", "ankara", "antalya", "artvin",
    "aydin", "balikesir", "bilecik", "bingol", "bitlis", "bolu", "burdur", "bursa",
    "canakkale", "cankiri", "corum", "denizli", "diyarbakir", "edirne", "elazig", "erzincan",
    "erzurum", "eskisehir", "gaziantep", "giresun", "gumushane", "hakkari", "hatay", "isparta",
    "mersin", "istanbul", "izmir", "kars", "kastamonu", "kayseri", "kirklareli", "kirsehir",
    "kocaeli", "konya", "kutahya", "malatya", "manisa", "kahramanmaras", "mardin", "mugla",
    "mus", "nevsehir", "nigde", "ordu", "rize", "sakarya", "samsun", "siirt",
    "sinop", "sivas", "tekirdag", "tokat", "trabzon", "tunceli", "sanliurfa", "usak",
    "van", "yozgat", "zonguldak", "aksaray", "bayburt", "karaman", "kirikkale", "batman",
    "sirnak", "bartin", "ardahan", "igdir", "yalova", "karabuk", "kilis", "osmaniye", "duzce"
]

In [10]:
def post_url(url):
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64)...",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)...",
    ]
    headers = {
        "User-Agent": random.choice(user_agents),
        "Accept-Language": "en-US,en;q=0.9",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9",
        "Referer": "https://www.google.com/",
        "Connection": "keep-alive"
    }
    session = requests.Session()
    response = session.get(url, headers=headers)
    soup=BeautifulSoup(response.content,"html.parser")
    return soup
def url(sehir,page):
    return f"https://www.arabam.com/ikinci-el/otomobil-{sehir}?view=List&page={page}"

In [11]:
for city in iller:
    fiyat2 = ""
    for page in range(1, 51):
        try:
            sehir = post_url(url(city, page))
            table = sehir.find("table", "table listing-table w100")
            links = table.find_all("tr", "listing-list-item should-hover bg-white")
            for link_item in links:
                link = link_item.find("a", "link-overlay")
                link2 = "https://www.arabam.com/" + link.get("href")
                detay = {}
                son = post_url(link2)
                baslik1 = son.find("div", "product-name-container")
                
                konum1 = son.find("span", "product-location")
                
                fiyat1 = son.find("div", "desktop-information-price")
                
                details = son.find("div", "product-properties-details linear-gradient")
                all_details = details.find_all("div", "property-item")
                
                try:
                    baslik = baslik1.text.strip()
                    konum = konum1.text.strip()
                    fiyat = fiyat1.text.strip()
                except:
                    pass
                
                detay = {
                    "İlan Tarihi": None,
                    "Marka": None,
                    "Seri": None,
                    "Model": None,
                    "Yıl": None,
                    "Kilometre": None,
                    "Vites Tipi": None,
                    "Yakıt Tipi": None,
                    "Kasa Tipi": None,
                    "Renk": None,
                    "Motor Hacmi": None,
                    "Motor Gücü": None,
                    "Çekiş": None,
                    "Araç Durumu": None,
                    "Ort. Yakıt Tüketimi": None,
                    "Yakıt Deposu": None,
                    "Boya-değişen": None,
                    "Takasa Uygun": None,
                    "Kimden": None
                }
                
                key = ""
                value = ""
                for detail_item in all_details[1:]:
                    key_div = detail_item.find("div", "property-key")
                    value_div = detail_item.find("div", "property-value")
                    try:
                        key = key_div.text.strip() if key_div else None
                        value = value_div.text.strip() if value_div else None
                    except:
                        pass
                
                    if key:
                        detay[key] = value
        
                aciklama = son.find("div", "tab-content-wrapper tab-description")
                pattern = r'(tramer(?:\skaydı)?|hasar(?:\skaydı)?)\s*[:\-]?\s*([\d.,]+)'
                matches = re.findall(pattern, aciklama.text, re.IGNORECASE | re.DOTALL)
                
                for label, fiyat_str in matches:
                    try:
                        fiyat2 = int(fiyat_str.replace('.', '').replace(',', ''))
                    except:
                        pass
                
                yeni_satir = pd.DataFrame([{
                    "baslik": baslik,
                    "konum": konum,
                    "fiyat": fiyat,
                    "ilan_tarihi": detay["İlan Tarihi"],
                    "marka": detay["Marka"],
                    "seri": detay["Seri"],
                    "model": detay["Model"],
                    "yil": detay["Yıl"],
                    "kilometre": detay["Kilometre"],
                    "vites_tipi": detay["Vites Tipi"],
                    "yakit_tipi": detay["Yakıt Tipi"],
                    "kasa_tipi": detay["Kasa Tipi"],
                    "renk": detay["Renk"],
                    "motor_hacmi": detay["Motor Hacmi"],
                    "motor_gucu": detay["Motor Gücü"],
                    "cekis": detay["Çekiş"],
                    "arac_durumu": detay["Araç Durumu"],
                    "ortalama_yakit_tuketimi": detay["Ort. Yakıt Tüketimi"],
                    "yakit_deposu": detay["Yakıt Deposu"],
                    "boya_degisen": detay["Boya-değişen"],
                    "takasa_uygun": detay["Takasa Uygun"],
                    "kimden": detay["Kimden"],
                    "tramer": fiyat2
                }])
                yeni_satir.to_csv("cars.csv", mode='a', header=False, index=False)
                detay = {}
                fiyat2 = ""
        except:
            pass